In [1]:
import websocket
import sys
# from websocket import create_connection

import simplejson as json

sys.path.append('../trainedModel_historical\MetaTrader5')


<>:7: SyntaxWarning: invalid escape sequence '\M'
<>:7: SyntaxWarning: invalid escape sequence '\M'
C:\Users\tshol\AppData\Local\Temp\ipykernel_40276\310442952.py:7: SyntaxWarning: invalid escape sequence '\M'
  sys.path.append('../trainedModel_historical\MetaTrader5')


In [2]:

# ws = websocket.create_connection("wss://api.tiingo.com/fx")

# subscribe = {
#         'eventName':'subscribe',
#         'authorization':'ebed6535a2acbe1a06b0057e8d1e8af61e5cefa7',
#         'eventData': {
#             'thresholdLevel': 5,
#             'tickers': ['eurusd']
#     }
# }

# ws.send(json.dumps(subscribe))
# while True:
#     print(ws.recv())

## __Receives data *by second*__

In [2]:
import websocket
import json
import pandas as pd
from datetime import datetime

# Create an empty DataFrame to store real-time data
df = pd.DataFrame(columns=['ticker', 'datetime', 'bidPrice', 'askPrice', 'midPrice'])

# WebSocket connection
ws = websocket.create_connection("wss://api.tiingo.com/fx")

# Subscription message
subscribe = {
    'eventName': 'subscribe',
    'authorization': 'ebed6535a2acbe1a06b0057e8d1e8af61e5cefa7',  # Replace with your actual API key
    'eventData': {
        'thresholdLevel': 5,
        'tickers': ['eurusd']
    }
}
# --------------------------------------------------
# # Send subscription message
# ws.send(json.dumps(subscribe))

# # Function to process received data and append it to DataFrame
# def process_message(message):
#     global df
#     data = json.loads(message)
    
#     # Only process price update messages (messageType == 'A')
#     if data.get('service') == 'fx' and data.get('messageType') == 'A':
#         fx_data = data['data']
        
#         # Extract relevant fields from the message
#         ticker = fx_data[1]  # Ticker (e.g., 'eurusd')
#         datetime_str = fx_data[2]  # Date as string
#         bid_price = fx_data[4]  # Bid price
#         ask_price = fx_data[6]  # Ask price
#         mid_price = fx_data[5]  # Mid price (calculated as (bid + ask) / 2)
        
#         # Convert datetime string to a proper datetime object (optional)
#         timestamp = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%f%z")
        
      
#         new_row = pd.DataFrame([{
#             'ticker': ticker,
#             'datetime': timestamp,
#             'bidPrice': bid_price,
#             'askPrice': ask_price,
#             'midPrice': mid_price
#         }])


#         # df = df.append(new_row, ignore_index=True)
#         df = pd.concat([df, new_row], ignore_index=True)
        

#         # Optionally, display the updated DataFrame or save to a file
#         print(df.tail(1))  # Display the last few rows

# # Main loop to receive data and process it
# while True:
#     message = ws.recv()
#     # print(message)
#     process_message(message)



## __Recieves data  *by minute*__

In [3]:
import pandas as pd
import json
from datetime import datetime
import pandas_ta as ta 


df = pd.DataFrame(columns=['ticker', 'datetime', 'bidPrice', 'askPrice', 'midPrice'])
last_minute = None
last_price = None


ws.send(json.dumps(subscribe))

# Function to process received data and append it to DataFrame
def process_message(message):
    global df, last_minute, last_price
    data = json.loads(message)
    
    # Only process price update messages (messageType == 'A')
    if data.get('service') == 'fx' and data.get('messageType') == 'A':
        fx_data = data['data']
        
        # Extract relevant fields from the message
        ticker = fx_data[1]  # Ticker (e.g., 'eurusd')
        datetime_str = fx_data[2]  # Date as string
        bid_price = fx_data[4]  # Bid price
        ask_price = fx_data[6]  # Ask price
        mid_price = fx_data[5]  # Mid price (calculated as (bid + ask) / 2)
        
        # # Convert datetime string to a proper datetime object
        # timestamp = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%f%z")
        # current_minute = timestamp.replace(second=0, microsecond=0)  # Get the minute

        # Convert datetime string to a proper datetime object (handle cases with or without microseconds)
        try:
            timestamp = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%f%z")  # With microseconds
        except ValueError:
            timestamp = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S%z")  # Without microseconds

        current_minute = timestamp.replace(second=0, microsecond=0)  # 

        # Check if we're still in the same minute
        if last_minute is None:
            last_minute = current_minute
        
        if current_minute != last_minute:
            # Append the last price of the previous minute
            new_row = {
                'ticker': ticker,
                'datetime': last_minute,
                'bidPrice': bid_price,  # you can keep bid/ask from the last second if you prefer
                'askPrice': ask_price,
                'midPrice': last_price
            }
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            
            if len(df) > 54:
                # Apply transformations and calculate technical indicators
                df = calculate_indicators(df)
            else:
                print(f" --------------- dataframe length : {len(df)} - not ready. ---------------------- ")

            
            print(df.tail(1))


            last_minute = current_minute
        

        last_price = mid_price

# Function to calculate technical indicators
def calculate_indicators(data):
    # Renaming columns for compatibility with indicators
    data['close'] = data['midPrice']
    data['high'] = data['askPrice']
    data['low'] = data['bidPrice']
    data.dropna(subset=['close'], inplace=True)




    data['SMA_10'] = ta.sma(data['close'], length=10)
    

    data['EMA_10'] = ta.ema(data['close'], length=10)
    
    # # Moving Average Convergence Divergence (MACD)
    # macd = ta.macd(data['close'])
    # data['MACD_Histogram'] = macd['MACDh_12_26_9']
    # Moving Average Convergence Divergence (MACD)
    macd = ta.macd(data['close'])
    
    # Safeguard to ensure MACD data is available before accessing columns
    if macd is not None:
        macd = macd.fillna(0)
        data['MACD_Histogram'] = macd['MACDh_12_26_9']
        data['MACD_Histogram'].fillna(0, inplace = True)
    else:
        print("xxxxxxxxx MACD is None  xxxxxxxxxxxxxxxxxxxx")
        macd = macd.fillna(0)
        data['MACD_Histogram'] = None  # Handle missing MACD values



    # # Safeguard to ensure MACD data is available before accessing columns
    # if macd is not None:
    #     # Fill missing values (NaN) in MACD columns before using them
    #     macd = macd.fillna(0)
    #     data['MACD_Histogram'] = macd['MACDh_12_26_9']
    # else:
    #     data['MACD_Histogram'] = None  # Handle missing MACD values


    data['RSI_14'] = ta.rsi(data['close'], length=14)
    

    data['Williams_%R'] = ta.willr(data['high'], data['low'], data['close'], length=14)
    

    data.drop(columns=['close', 'high', 'low'], inplace=True)

    return data

# Main loop to receive data and process it
while True:
    message = ws.recv()
    process_message(message)


C:\Users\tshol\AppData\Local\Temp\ipykernel_40276\1201687106.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


 --------------- dataframe length : 1 - not ready. ---------------------- 
   ticker                  datetime  bidPrice   askPrice  midPrice
0  eurusd 2024-10-04 20:21:00+00:00   1.09764  1000000.0   1.09768
 --------------- dataframe length : 2 - not ready. ---------------------- 
   ticker                  datetime  bidPrice   askPrice  midPrice
1  eurusd 2024-10-04 20:22:00+00:00   1.09763  1000000.0  1.097685
 --------------- dataframe length : 3 - not ready. ---------------------- 
   ticker                  datetime  bidPrice   askPrice  midPrice
2  eurusd 2024-10-04 20:23:00+00:00   1.09763  1000000.0   1.09768
 --------------- dataframe length : 4 - not ready. ---------------------- 
   ticker                  datetime  bidPrice   askPrice  midPrice
3  eurusd 2024-10-04 20:24:00+00:00   1.09764  1000000.0  1.097695
 --------------- dataframe length : 5 - not ready. ---------------------- 
   ticker                  datetime  bidPrice   askPrice  midPrice
4  eurusd 2024-10-04 2

KeyboardInterrupt: 

In [3]:
# # Global variable to track the last price of each minute
# last_minute = None
# last_price = None

# # Send subscription message
# ws.send(json.dumps(subscribe))

# # Function to process received data and append it to DataFrame
# def process_message(message):
#     global df, last_minute, last_price
#     data = json.loads(message)
    
#     # Only process price update messages (messageType == 'A')
#     if data.get('service') == 'fx' and data.get('messageType') == 'A':
#         fx_data = data['data']
        
#         # Extract relevant fields from the message
#         ticker = fx_data[1]  # Ticker (e.g., 'eurusd')
#         datetime_str = fx_data[2]  # Date as string
#         bid_price = fx_data[4]  # Bid price
#         ask_price = fx_data[6]  # Ask price
#         mid_price = fx_data[5]  # Mid price (calculated as (bid + ask) / 2)
        
#         # Convert datetime string to a proper datetime object
#         timestamp = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%f%z")
#         current_minute = timestamp.replace(second=0, microsecond=0)  # Get the minute

#         # Check if we're still in the same minute
#         if last_minute is None:
#             last_minute = current_minute
        
#         if current_minute != last_minute:
#             # Append the last price of the previous minute
#             new_row = {
#                 'ticker': ticker,
#                 'datetime': last_minute,
#                 'bidPrice': bid_price,  # you can keep bid/ask from the last second if you prefer
#                 'askPrice': ask_price,
#                 'midPrice': last_price
#             }
#             df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
#             print(df.tail(1))  # Display the last row

#             # Update to the new minute
#             last_minute = current_minute
        
#         # Store the last midPrice of the current minute
#         last_price = mid_price

# # Main loop to receive data and process it
# while True:
#     message = ws.recv()
#     process_message(message)


C:\Users\tshol\AppData\Local\Temp\ipykernel_11120\1458787691.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


   ticker                  datetime  bidPrice   askPrice  midPrice
0  eurusd 2024-09-30 19:24:00+00:00   1.11276  1000000.0  1.112795
   ticker                  datetime  bidPrice   askPrice  midPrice
1  eurusd 2024-09-30 19:25:00+00:00   1.11277  1000000.0  1.112815
   ticker                  datetime  bidPrice   askPrice  midPrice
2  eurusd 2024-09-30 19:26:00+00:00   1.11277  1000000.0   1.11282


KeyboardInterrupt: 